# ETL III Pair 24/08

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [16]:
import pandas as pd
import requests
from datetime import datetime

In [17]:
class EnergiaPoblacion:
    
    def __init__(self, start_year = 2011, end_year = 2022, path_to_save = "data"):
        self.start_year = start_year
        self.end_year = end_year
        self.path_to_save = path_to_save
        self.ccaa = {
            'Ceuta': 8744,
            'Melilla': 8745,
            'Andalucía': 4,
            'Aragón': 5,
            'Cantabria': 6,
            'Castilla - La Mancha': 7,
            'Castilla y León': 8,
            'Cataluña': 9,
            'País Vasco': 10,
            'Principado de Asturias': 11,
            'Comunidad de Madrid': 13,
            'Comunidad Foral de Navarra': 14,
            'Comunitat Valenciana': 15,
            'Extremadura': 16,
            'Galicia': 17,
            'Illes Balears': 8743,
            'Canarias': 8742,
            'Región de Murcia': 21,
            'La Rioja': 20
        }
        
    def info_energia_nacional(self):
        """Obtener la información de la evolución de las energías renovables y no renovables en el periodo de tiempo solicitado.

        Returns:
            dataframe: dataframe con la información de todos los años solicitados.
        """
        print(f"Ejecutando función 'info_energia_nacional' para el período de tiempo {self.start_year}-{self.end_year}")
        df_final = pd.DataFrame()
        for year in range(self.start_year, self.end_year + 1):
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day"
            response = requests.get(url=url)
            status_code = response.status_code
            print(f"{year} : Respuesta de la petición: {status_code}")
                    
            if status_code == 200:
                response_data = response.json()['included']
                len_response_data = len(response_data)
                
                for i in range(len_response_data):
                    attributes_data = response_data[i]['attributes']
                    type_energy = attributes_data['type']
                    values_data = attributes_data['values']
                    df = pd.json_normalize(values_data)
                    df["tipo_energia"] = type_energy
                    df_final = pd.concat([df_final, df], axis = 0)
            else:
                continue
        return df_final.reset_index(drop = True)
    
    def info_energia_ccaa(self):
        """Obtener la información de la evolución de las energías renovables y no renovables de todas las CCAA en el periodo de tiempo solicitado.

        Returns:
            dataframe: dataframe con la información de todas las CCAA y año solicitado.
        """
        print(f"Ejecutando función 'info_energia_ccaa' para el período de tiempo {self.start_year}-{self.end_year}")
        df_final = pd.DataFrame()
        for year in range(self.start_year, self.end_year + 1):
            for k_nombre_ccaa, v_id_ccaa in self.ccaa.items():
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day&geo_limit=ccaa&geo_ids={v_id_ccaa}"
                response = requests.get(url=url)
                status_code = response.status_code
                print(f"{year} - {k_nombre_ccaa} : Respuesta de la petición: {status_code}")
                if status_code == 200:
                    response_data = response.json()['included']
                    len_response_data = len(response_data)
                    
                    for i in range(len_response_data):
                        attributes_data = response_data[i]['attributes']
                        type_energy = attributes_data['type']
                        values_data = attributes_data['values']
                        df = pd.json_normalize(values_data)
                        df["tipo_energia"] = type_energy
                        df["comunidad"] = k_nombre_ccaa
                        df["id_comunidad"] = v_id_ccaa
                        df_final = pd.concat([df_final, df], axis = 0)
                else:
                    continue
        return df_final.reset_index(drop = True)
            
    def drop_column(self, df, col):
        """Eliminar la columna solicitada en el dataframe aportado.

        Args:
            df (dataframe): dataframe utilizado.
            col (str): nombre de la columna.

        Returns:
            dataframe: dataframe utilizado sin la columna.
        """
        print(f"Eliminando columna '{col}'")
        return df.drop(columns = [col], axis = 0, inplace = True )
        
    def round_two(self, df, columns):
        """Redondear los valores a dos decimales.

        Args:
            df (dataframe): dataframe utilizado.
            columns (list): lista con las columnas a tratar.

        Returns:
            dataframe: dataframe con los valores redondeados a 2 decimales
        """
        for col in columns:
            print(f"Redondeando columna '{col}' a 2 decimales")
            df[col] = df[col].round(2)
    
    def create_datetime_col(self, df, datetimeCol, newCol):
        """Crear una columna de tipo datetime con el nombre aportado a partir de una columna de tipo object

        Args:
            df (dataframe): dataframe utilizado.
            datetimeCol (str): nombre de la columna origen
            newCol (str): nombre de la nueva columna
        """
        print(f"Creando columna '{newCol}' a partir de la columna '{datetimeCol}'")
        df[newCol] = df[datetimeCol].str.split("T", n = 1, expand = True).get(0).astype("datetime64", errors = "ignore")
        df[newCol] = pd.to_datetime(df[newCol])

    def open_csv(self, path, file_name, years = False):
        """Abrir el archivo csv en la ruta especificada con el nombre aportado.

        Args:
            path (str): ruta donde se encuentra el fichero
            file_name (str): nombre del archivo
            years (bool, optional): Especificamos si debemos buscar el fichero con el intervalo de tiempo deseado al generar la clase. Defaults to False.

        Returns:
            dataframe: dataframe generado a partir del archivo csv leído
        """
        if years:
            path_file = f"{path}/{file_name}_{self.start_year}_{self.end_year}.csv"
        else:
            path_file = f"{path}/{file_name}.csv"
        
        print(f"Leyendo: {path_file}")  
        return pd.read_csv(path_file)
    
    def open_pickle(self, path, file_name, years = False):
        """Abrir el archivo pickle en la ruta especificada con el nombre aportado.

        Args:
            path (str): ruta donde se encuentra el fichero
            file_name (str): nombre del archivo
            years (bool, optional): Especificamos si debemos buscar el fichero con el intervalo de tiempo deseado al generar la clase. Defaults to False.

        Returns:
            dataframe: dataframe generado a partir del archivo pickle leído
        """
        if years:
            path_file = f"{path}/{file_name}_{self.start_year}_{self.end_year}.pkl"
        else:
            path_file = f"{path}/{file_name}.pkl"
            
        print(f"Leyendo: {path_file}")  
        return pd.read_pickle(path_file)
           
    def save_to_csv(self, df, file_name, years = False):
        """Guardar el dataframe aportado como un fichero csv.

        Args:
            df (dataframe): dataframe utilizado.
            file_name (str): nombre del archivo que queremos generar y guardar.
            years (bool, optional): Especificamos si debemos guardar el fichero con el intervalo de tiempo deseado al generar la clase. Defaults to False.
        """
        if years:
            path_file = f"{self.path_to_save}/{file_name}_{self.start_year}_{self.end_year}.csv"
        else:
            path_file = f"{self.path_to_save}/{file_name}.csv"
        
        print(f"Guardando en: {path_file}")  
        df.to_csv(path_file)
        
        
    def save_to_pickle(self, df, file_name, years = False):
        """Guardar el dataframe aportado como un fichero pickle.

        Args:
            df (dataframe): dataframe utilizado.
            file_name (str): nombre del archivo que queremos generar y guardar.
            years (bool, optional): Especificamos si debemos guardar el fichero con el intervalo de tiempo deseado al generar la clase. Defaults to False.
        """
        if years:
            path_file = f"{self.path_to_save}/{file_name}_{self.start_year}_{self.end_year}.pkl"
        else:
            path_file = f"{self.path_to_save}/{file_name}.pkl"
            
        print(f"Guardando en: {path_file}")  
        df.to_pickle(path_file)

## Aplicamos clase

### Datos Población Comunidades

In [18]:
clase_pair3_poblacion_ccaa = EnergiaPoblacion()
df_pair3_poblacion_ccaa = clase_pair3_poblacion_ccaa.open_csv("data","poblacion_comunidades",False)
df_pair3_poblacion_ccaa.head()

Leyendo: data/poblacion_comunidades.csv


,Unnamed: 0,Comunidad,indice,Comunidades_y_Ciudades_Autónomas,Año,Total
0,0,Ceuta,8744,Ceuta,2022,83.117
1,1,Ceuta,8744,Ceuta,2021,83.517
2,2,Ceuta,8744,Ceuta,2020,84.202
3,3,Ceuta,8744,Ceuta,2019,84.777
4,4,Ceuta,8744,Ceuta,2018,85.144


In [19]:
clase_pair3_poblacion_ccaa.drop_column(df_pair3_poblacion_ccaa, "Comunidades_y_Ciudades_Autónomas")
df_pair3_poblacion_ccaa.columns

Eliminando columna 'Comunidades_y_Ciudades_Autónomas'


Index(['Unnamed: 0', 'Comunidad', 'indice', 'Año', 'Total'], dtype='object')

In [20]:
clase_pair3_poblacion_ccaa.save_to_csv(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", False)
clase_pair3_poblacion_ccaa.save_to_pickle(df_pair3_poblacion_ccaa, "pairETL3_poblacion_comunidades", False)

Guardando en: data/pairETL3_poblacion_comunidades.csv
Guardando en: data/pairETL3_poblacion_comunidades.pkl


### Datos Energia Nacional

In [21]:
clase_pair3_energia_nacional = EnergiaPoblacion(start_year=2011, end_year=2022)
df_pair3_energia_nacional = clase_pair3_energia_nacional.info_energia_nacional()
df_pair3_energia_nacional.head()

Ejecutando función 'info_energia_nacional' para el período de tiempo 2011-2022


2011 : Respuesta de la petición: 200
2012 : Respuesta de la petición: 200
2013 : Respuesta de la petición: 200
2014 : Respuesta de la petición: 200
2015 : Respuesta de la petición: 200
2016 : Respuesta de la petición: 200
2017 : Respuesta de la petición: 200
2018 : Respuesta de la petición: 200
2019 : Respuesta de la petición: 200
2020 : Respuesta de la petición: 200
2021 : Respuesta de la petición: 200
2022 : Respuesta de la petición: 200


,value,percentage,datetime,tipo_energia
0,182024.9820,0.290121,2011-01-01T00:00:00.000+01:00,Renovable
1,265705.8895,0.387418,2011-01-02T00:00:00.000+01:00,Renovable
2,249489.5735,0.295983,2011-01-03T00:00:00.000+01:00,Renovable
3,270981.7130,0.310937,2011-01-04T00:00:00.000+01:00,Renovable
4,379833.0150,0.452824,2011-01-05T00:00:00.000+01:00,Renovable


In [22]:
clase_pair3_energia_nacional.round_two(df_pair3_energia_nacional, ["value", "percentage"])

df_pair3_energia_nacional.head()

Redondeando columna 'value' a 2 decimales
Redondeando columna 'percentage' a 2 decimales


,value,percentage,datetime,tipo_energia
0,182024.98,0.29,2011-01-01T00:00:00.000+01:00,Renovable
1,265705.89,0.39,2011-01-02T00:00:00.000+01:00,Renovable
2,249489.57,0.30,2011-01-03T00:00:00.000+01:00,Renovable
3,270981.71,0.31,2011-01-04T00:00:00.000+01:00,Renovable
4,379833.02,0.45,2011-01-05T00:00:00.000+01:00,Renovable


In [23]:
clase_pair3_energia_nacional.create_datetime_col(df_pair3_energia_nacional, "datetime", "fecha")
df_pair3_energia_nacional.head()

Creando columna 'fecha' a partir de la columna 'datetime'


,value,percentage,datetime,tipo_energia,fecha
0,182024.98,0.29,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01
1,265705.89,0.39,2011-01-02T00:00:00.000+01:00,Renovable,2011-01-02
2,249489.57,0.30,2011-01-03T00:00:00.000+01:00,Renovable,2011-01-03
3,270981.71,0.31,2011-01-04T00:00:00.000+01:00,Renovable,2011-01-04
4,379833.02,0.45,2011-01-05T00:00:00.000+01:00,Renovable,2011-01-05


In [24]:
clase_pair3_energia_nacional.drop_column(df_pair3_energia_nacional, "datetime")
df_pair3_energia_nacional.head()

Eliminando columna 'datetime'


,value,percentage,tipo_energia,fecha
0,182024.98,0.29,Renovable,2011-01-01
1,265705.89,0.39,Renovable,2011-01-02
2,249489.57,0.30,Renovable,2011-01-03
3,270981.71,0.31,Renovable,2011-01-04
4,379833.02,0.45,Renovable,2011-01-05


In [25]:
clase_pair3_energia_nacional.save_to_csv(df_pair3_energia_nacional, "pairETL3_energia_nacional", False)
clase_pair3_energia_nacional.save_to_csv(df_pair3_energia_nacional, "pairETL3_energia_nacional", True)
clase_pair3_energia_nacional.save_to_pickle(df_pair3_energia_nacional, "pairETL3_energia_nacional", False)
clase_pair3_energia_nacional.save_to_pickle(df_pair3_energia_nacional, "pairETL3_energia_nacional", True)

Guardando en: data/pairETL3_energia_nacional.csv
Guardando en: data/pairETL3_energia_nacional_2011_2022.csv
Guardando en: data/pairETL3_energia_nacional.pkl
Guardando en: data/pairETL3_energia_nacional_2011_2022.pkl


### Datos Energia CCAA

In [26]:
clase_pair3_energia_ccaa = EnergiaPoblacion(2011,2022)
df_pair3_energia_ccaa = clase_pair3_energia_ccaa.info_energia_ccaa()

Ejecutando función 'info_energia_ccaa' para el período de tiempo 2011-2022


2011 - Ceuta : Respuesta de la petición: 200
2011 - Melilla : Respuesta de la petición: 200
2011 - Andalucía : Respuesta de la petición: 200
2011 - Aragón : Respuesta de la petición: 200
2011 - Cantabria : Respuesta de la petición: 200
2011 - Castilla - La Mancha : Respuesta de la petición: 200
2011 - Castilla y León : Respuesta de la petición: 200
2011 - Cataluña : Respuesta de la petición: 200
2011 - País Vasco : Respuesta de la petición: 200
2011 - Principado de Asturias : Respuesta de la petición: 200
2011 - Comunidad de Madrid : Respuesta de la petición: 200
2011 - Comunidad Foral de Navarra : Respuesta de la petición: 200
2011 - Comunitat Valenciana : Respuesta de la petición: 200
2011 - Extremadura : Respuesta de la petición: 200
2011 - Galicia : Respuesta de la petición: 200
2011 - Illes Balears : Respuesta de la petición: 200
2011 - Canarias : Respuesta de la petición: 200
2011 - Región de Murcia : Respuesta de la petición: 200
2011 - La Rioja : Respuesta de la petición: 200
2

In [27]:
clase_pair3_energia_ccaa.round_two(df_pair3_energia_ccaa, ["value", "percentage"])
df_pair3_energia_ccaa.head()

Redondeando columna 'value' a 2 decimales
Redondeando columna 'percentage' a 2 decimales


,value,percentage,datetime,tipo_energia,comunidad,id_comunidad
0,460.05,1.0,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
1,462.98,1.0,2011-01-02T00:00:00.000+01:00,No renovable,Ceuta,8744
2,537.07,1.0,2011-01-03T00:00:00.000+01:00,No renovable,Ceuta,8744
3,539.19,1.0,2011-01-04T00:00:00.000+01:00,No renovable,Ceuta,8744
4,536.03,1.0,2011-01-05T00:00:00.000+01:00,No renovable,Ceuta,8744


In [28]:
clase_pair3_energia_ccaa.create_datetime_col(df_pair3_energia_ccaa, "datetime", "fecha")
df_pair3_energia_ccaa.head()

Creando columna 'fecha' a partir de la columna 'datetime'


,value,percentage,datetime,tipo_energia,comunidad,id_comunidad,fecha
0,460.05,1.0,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-01
1,462.98,1.0,2011-01-02T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-02
2,537.07,1.0,2011-01-03T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-03
3,539.19,1.0,2011-01-04T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-04
4,536.03,1.0,2011-01-05T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-05


In [29]:
clase_pair3_energia_ccaa.drop_column(df_pair3_energia_ccaa, "datetime")
df_pair3_energia_ccaa.head()

Eliminando columna 'datetime'


,value,percentage,tipo_energia,comunidad,id_comunidad,fecha
0,460.05,1.0,No renovable,Ceuta,8744,2011-01-01
1,462.98,1.0,No renovable,Ceuta,8744,2011-01-02
2,537.07,1.0,No renovable,Ceuta,8744,2011-01-03
3,539.19,1.0,No renovable,Ceuta,8744,2011-01-04
4,536.03,1.0,No renovable,Ceuta,8744,2011-01-05


In [30]:
clase_pair3_energia_ccaa.save_to_csv(df_pair3_energia_ccaa, "pairETL3_energia_ccaa", False)
clase_pair3_energia_ccaa.save_to_csv(df_pair3_energia_ccaa, "pairETL3_energia_ccaa", True)
clase_pair3_energia_ccaa.save_to_pickle(df_pair3_energia_ccaa, "pairETL3_energia_ccaa", False)
clase_pair3_energia_ccaa.save_to_pickle(df_pair3_energia_ccaa, "pairETL3_energia_ccaa", True)

Guardando en: data/pairETL3_energia_ccaa.csv


Guardando en: data/pairETL3_energia_ccaa_2011_2022.csv
Guardando en: data/pairETL3_energia_ccaa.pkl
Guardando en: data/pairETL3_energia_ccaa_2011_2022.pkl
